# Operator Estimation

In [ ]:
import numpy as np
from pyquil import Program, get_qc
from pyquil.gates import *
qc = get_qc('2q-qvm', noisy=True)

## Construct a `ObservablesExperiment` and turn it into a list of `Programs`


In [ ]:
from pyquil.paulis import *
from forest.benchmarking.operator_estimation import ExperimentSetting, plusX, ObservablesExperiment, generate_experiment_programs
q = 0
# make ExperimentSettings for tomographing the plus state
expt_settings = [ExperimentSetting(plusX(q), pt) for pt in [sX(q), sY(q), sZ(q)]]
print('ExperimentSettings:\n==================\n',expt_settings,'\n')

# make an ObservablesExperiment
obs_expt = ObservablesExperiment(expt_settings, program=Program())
print('ObservablesExperiment:\n======================\n',obs_expt,'\n')

# convert it to a list of programs and a list of qubits for each program
expt_progs, qubits = generate_experiment_programs(obs_expt)
print('Programs and Qubits:\n======================\n')
for prog, qs in zip(expt_progs, qubits):
    print(prog, qs)


## At this point we can run directly or first symmetrize

## 1) run programs directly

In [ ]:
from forest.benchmarking.operator_estimation import _measure_bitstrings
# this is a simple wrapper that adds measure instructions for each qubit in qubits and runs each program.
results = _measure_bitstrings(qc, expt_progs, qubits, num_shots=5)
for bits in results:
    print(bits)

## 2) First symmetrize, then run

A symmetrization method should return  
1. a list of programs 
2. a list of measurement qubits associated with each program
3. a list of bits/bools indicating which qubits where flipped before measurement for each program
4. a list of ints indicating the group of results to which each program's outputs should be re-assigned after correction

In [ ]:
from forest.benchmarking.operator_estimation import exhaustive_symmetrization, consolidate_symmetrization_outputs
symm_progs, symm_qs, flip_arrays, groups = exhaustive_symmetrization(expt_progs, qubits)
print('2 symmetrized programs for each original:\n======================\n')
for prog in symm_progs:
    print(prog)

# now these programs can be run as above
results = _measure_bitstrings(qc, symm_progs, symm_qs, num_shots=5)

# we now need to consolidate these results using the information in flip_arrays and groups 
results = consolidate_symmetrization_outputs(results, flip_arrays, groups)

print('After consolidating we have twice the number of (symmetrized) shots as above:\n======================\n')
# we now have twice the number of symmetrized shots
for bits in results:
    print(bits)

## Now we can translate our bitarray shots into ExperimentResults with expectation values

In [ ]:
from forest.benchmarking.operator_estimation import shots_to_obs_moments, ExperimentResult

expt_results_as_obs = []
# we use the settings from the ObservableExperiment to label our results
for bitarray, meas_qs, settings in zip(results, qubits, obs_expt):
    # settings is a list of settings run simultaneously for a given program;
    # in our case, we just had one setting per program so this isn't strictly uncessary
    for setting in settings:
        observable = setting.observable # get the PauliTerm
        
        # Obtain statistics from result of experiment
        obs_mean, obs_var = shots_to_obs_moments(bitarray, meas_qs, observable)
        
        expt_results_as_obs.append(ExperimentResult(setting, obs_mean, std_err = np.sqrt(obs_var),
                                                    total_counts = len(bitarray)))

for res in expt_results_as_obs:
    print(res)

## All of the above is wrapped in a convenience function estimate_observables

In [ ]:
from forest.benchmarking.operator_estimation import estimate_observables

for res in estimate_observables(qc, obs_expt, num_shots=50, symmetrization_method = exhaustive_symmetrization):
    print(res)


## Finally after obtaining our results we can choose to calibrate our observables

This uses essentially the same workflow as above except that the programs are generated for each observable via 
`get_calibration_program`. I'll only demonstrate the convience wrapper below.

In [ ]:
from forest.benchmarking.operator_estimation import calibrate_observable_estimates

for cal_res in calibrate_observable_estimates(qc, expt_results_as_obs, 
                                              symmetrization_method=exhaustive_symmetrization,
                                             num_shots=100):
    print(cal_res)
    print(f'Original expectation was {cal_res.raw_expectation}')
    print(f'Observable cal expectation was {cal_res.calibration_expectation} \n')